<div>
<h1>Autocorrelation Phase Matrix</h1> 
</div>

Which are the weaknesses of autocorrelation-based tempogram?

It assumes that beats are always equally spaced.

What if:
* Beat can shift both in **time** (e.g. between different levels of the metrical hierarchy) 
* Beat can shift in **phase** (e.g. from unsyncopated to syncopated).
* Beat can **change** in the middle of a piece, and it will corresponds to when the meter and tempo shifts are encountered
* Polyrhythms are present (hemiola, cross-rhythm,...) ?

An ideal beat estimation process should consider multiple hypotheses about beat in parallel and should allow to switch between these hypotheses.

# Autocorrelation Phase Matrix

Let's consider again a given **windowed** novelty function of **finite length** $N$ $\Delta_{w,n}(m):\mathbb{Z}\to\mathbb{R}$, defined as we did for autocorrelation tempogram definition.


For notation simplicity, let's discard $n$ and $w$ parameters (window index and window type parameter) and consider just one window of the novelty function that we will indicate as $\Delta(m)$ of length $N$. The length of the windowed signal $N$ must be long enough for analysing all the possible tempos 
\begin{equation}
    1\leq  k_{\text{min}} \leq k_{\text{max}} \leq N-1.
\end{equation}


Then, the **autocorrelation phase matrix (APM)** will be defined as 

$$
P(\phi, k) = \sum_{i=0}^{\lceil\frac{N -\phi}{k}\rceil} \Delta(k i + \phi)\Delta(k(i+1)+\phi)
$$

where $k$ is the considered lag and $\phi$ is the phase.

Phase is constrained such that $\phi < k$, hence APM is a **triangular matrix**.

For each lag $k$ of interest, the APM stores intermediate results of autocorrelation in a vector of length $k$ such that the results of the dot product from the autocorrelation are wrapped into the APM row by their phase $\phi$. In this way APM preserves the distribution of autocorrelation energy in the **phase space**.


Let's see an image example:

<img src="data/img/apm_1.png" width="800px" align="center" alt="APM first example">
<img src="data/img/apm_2.png" width="800px" align="center" alt="APM second example">
<img src="data/img/apm_3.png" width="800px" align="center" alt="APM third example">


Let's define also a counter matrix $C$ that allows to "normalize" the APM:

$$
C(\phi, k)=\frac{N-\phi}{k}
$$

Hence unbiased APM can be obtained as element-wise division of $P(\phi, k)$ and $C(\phi, k)$: 


$$
\hat{P}(\phi, k) = \frac{P(\phi, k)}{C(\phi, k)}
$$



The algorithm can be expressed in pseudocode as:


<img src="data/img/pseudocode.png" width="800px" align="left" alt="Pseudocode for APM computation">



### Autocorrelation from APM

Autocorrelation for a window $a(k)$ can be obtained by summing all phase values for each lag, i.e. summing the rows of the APM $P(k, \phi)$.

\begin{equation}
\mathcal{A}(k) = \sum_{i=0}^{k-1} P(k, i)
\end{equation}




## Implementation

In [1]:
import numpy as np
import librosa
import os
import matplotlib.pyplot as plt
import IPython.display as ipd

Let's define a **novelty function** (it's different from the novelty functions we analyized in class, so let's skip this)

In [2]:
def a_novelty_function(x, Fs=1, N=1024, H=512):
    """Compute complex-domain novelty function
    """     
    X = librosa.stft(x, n_fft=N, hop_length=H, win_length=N, window='hanning')
    Fs_feature = Fs/H
    mag = np.abs(X)
    
    phase = np.angle(X)/(2*np.pi)
    
    unwr_phase = np.zeros_like(X, dtype=float);
    for i in np.arange(X.shape[1]):
        unwr_phase[:,i] = np.unwrap( np.angle(X[:,i]) )
    
    def warp(x, low, interval):
        return np.remainder(x - low, interval) + low
    
    phase_shift = unwr_phase[:,2:] - 2*unwr_phase[:,1:-1] + unwr_phase[:,0:-2]
    phase_shift =  warp(phase_shift, -np.pi, 2*np.pi)
    
    
    amp_pred = mag[:,1:-1]
    amp_true = mag[:,2:]
    
    novelty_complex = (amp_pred**2 + amp_true**2 - 2 * amp_pred * amp_true * np.cos(phase_shift))
    
    # Half wave rectification
    novelty_complex[novelty_complex<0]=0
    
    novelty_complex = np.sqrt(novelty_complex)
    
    novelty_complex = np.sum(novelty_complex, axis=0)
    novelty_complex = np.concatenate((novelty_complex, np.array([0, 0])))
    return novelty_complex, Fs_feature


### Compute novelty function

### Compute APM

### In practice, how do we estimate tempo features from the APM?

We need to look at **persistent peaks** in the APM. Each peak corresponds to a lag $k$ and phase $\phi$ value and they identify one of the internal subdivision of the considered rhythm.

# Decomposition of APM matrix

Each element of the autocorrelation phase matrix $P(k,\phi)$ can be rewritten "unrolling" the sum:


\begin{equation}
    P(\phi, k) = \Delta(\phi)\Delta(k+\phi)+\Delta(k+\phi)\Delta(2k+\phi)+...+\Delta((K-2)k+\phi)\Delta((K-1)k+\phi)
\end{equation} 

where $K = \Big\lceil\frac{N -\phi}{k}\Big\rceil$ is the number of samples available in a $N$-long sequence for the pair $(\phi, k)$ (i.e. the number of elements in the summation). 

Looking at the first definition of APM, the first element corresponds to index $i=0$ in the summation, the second element corresponds to index $i=1$ in the summation, and so on...


This operation can be seen as the dot product of two vectors of length $K-1$:


\begin{equation} 
    P(\phi, k) = \Delta_1(\phi, k) \cdot \Delta_2(\phi, k)
\end{equation}
where 

\begin{equation}
     \Delta_1(\phi, k) = [\Delta(\phi), \Delta(\phi+k),...,\Delta(\phi+(K-2)k] \\
     \Delta_2(\phi, k) = [\Delta(\phi+k), \Delta(\phi+2k),...,\Delta(\phi+(K-1)k]
\end{equation}


Now consider two matrices, $D_1(\phi, k)$ and $D_2(\phi, k) $, where the element $(i,j)$ is defined as:


\begin{equation}
    D_1(\phi, k)_{i,j} = 
    \begin{cases} 
      1 &   \text{if} \ j =\phi+ki \\
      0 &   \text{else}
   \end{cases}
\end{equation}

\begin{equation}
   D_2(\phi, k)_{i,j} = 
    \begin{cases} 
      1 &   \text{if} \ j =\phi+k(i+1) \\
      0 &   \text{else}
   \end{cases}
\end{equation}

Please note that both $D_1(\phi, k)$ and $D_2(\phi, k)$ are defined for a specific couple of values of lag $k$ and phase $\phi$.


Then the two vectors, $x_1$ and $x_2$ can be expressed as:

\begin{equation}
    x_1(\phi, k) = [x(\phi), x(\phi+k),...,x(\phi+(K-2)k] = D_1 (\phi, k) x \\
    x_2(\phi, k) = [x(\phi+k), x(\phi+2k),...,x(\phi+(K-1)k] = D_2(\phi, k) x
\end{equation}

where $ x = [x(0), x(1), ... x(N)] $.


Finally the autocorrelation phase matrix can be reformulated as:
\begin{equation}
    P(\phi, k) = [D_1(\phi, k) x ] \cdot [D_2(\phi, k) x]
\end{equation}


## Time instant influence

Let's assume our onset novelty function is made of one single impulse signal:


\begin{equation}
    e_\hat{t}(t) = \begin{cases}
        1 & t = \hat{t}\\
        0 & \text{else}
        \end{cases}
\end{equation}

then, given the definition, $P(\phi, k) = 0$ for all $k$, $\phi$ values.

However, the fact that $P(\phi, k)$ is equal 0 **does not implicates** that each element of the just defined decomposition is equal 0. 

We can exploit the decomposition of $P(\phi, k)$ in this special case to give an answer to a question: which elements of $P(\phi, k)$ are "affected" by the value in $t=\hat{t}$? Meaning, for which couples of $(\phi, k)$ I am selecting $x(\hat{t})$ in the summation?

Think of it as an **inverse mapping** from lag-phase domain to time domain


Hence, considering  $P(\phi, k) = [D_1(\phi, k) e_\hat{t} ] \cdot [D_2(\phi, k) e_\hat{t}]$, the couples $(\phi, k)$ that satisfy:

\begin{equation}
    \sum_i D_1(\phi, k)e_\hat{t} \neq 0 \quad \text{or} \quad \sum_i D_2(\phi, k)e_\hat{t} \neq 0
\end{equation}


create a **set** which tell us the corrispondence between $(\phi, k)$ and $\hat{t}$. We call this set **influence of t** and it is indicated with $\mathbf{I_\hat{t}}$.

Note that the previous formulation is equivalent to
\begin{equation}
    \mathbf{I_\hat{t}}: \Big\{ (\phi, k) \quad \Big| \quad \hat{t} = \phi + ki \ \quad i=0,1,...,K-1 \Big\}
\end{equation}


We can create a **binary mask** in the $(\phi, k)$ space equal to 1 if $(\phi, k) \in \mathbf{I_\hat{t}}$ and equal to 0 otherwise.

In [8]:
def influence(t, lags, N):
    
    return 

As can be seen, in the $(k,\phi)$ space, $\hat{t}$ influence only certain **segments** of $P(\phi, k)$, in particular all the lines 


\begin{equation}
    k = -\frac{1}{i}\phi + \frac{\hat{t}}{i}
\end{equation}


with $i = \{0, 1, ..., K-1\}$. In particular $i=0$ corresponds to the vertical line.
(Think to the equation of a line where $k$ is the $y$ and $\phi$ is the $x$)


This is a beam of lines with center $(k_0,\phi_0)$. Therefore, for each slope $i$, it will be true that:
\begin{equation}
    i k_0 = -\phi_0 + \hat{t} \ \ \forall i
\end{equation}

and the values for the center of the beam are:
\begin{equation}
    k_0 = 0\\
    \phi_0 = \hat{t}
\end{equation}


The beam of lines correspondent is shown in this figure:
<img src="data/img/influence.png" width="600px" align="center" alt="APM first example">


### Let's consider two non zero points

We can now extend the previous considerations to the case of **two nonzero instants** $t_1, t_2$ in the input signal $x(j)$.
The big difference is that now some entries of  $P(\phi, k)$ will be nonzero. 
For the reasons explained before, this will happen for all the values of $(\phi, k)$ belonging to the intersection of $\mathbf{I_{t1}}$ and $\mathbf{I_{t2}}$

$$
\Big\{ (\phi, k) \in I_{t1} \cap I_{t2} \Big \}
$$ 


This is true at the intersections of the two beams, one centered in $(t1, 0)$ and the other in $(t2, 0)$.
Let's visualize these two beams in the $\phi$-$k$ space:

<img src="data/img/influence_2.png" width="600px" align="center" alt="APM first example">



What happens to the APM?

As expected, in the autocorrelation matrix $P(\phi, k)$ only intersections where both coordinates are integer can be observed. In order to show all the intersection we have to smooth the peaks at $t1$ and $t2$ and, to better show the lines in the beam, we are going to add a constant floor to the novelty function.